# Computing gradients 

In [1]:
import numpy as np 
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.layouts import row, column
output_notebook()

Loading BokehJS ...

Let's start by evaluating the difference between three ways of computing the derivatives of a given continuous function: 
1. closed-form 
2. symbolic computation
3. finite-difference 
4. automatic differentiation (à la autograd)

In [2]:
import numdifftools as nd
import autograd as ad
import autograd.numpy as auto_np
def sigmoid(x, derivative: bool = False):
  if not derivative:
    x = auto_np.clip(x, -30, 30)
    return auto_np.where(x >= 0, 1 / (1 + auto_np.exp(-x)), auto_np.exp(x) / (1 + auto_np.exp(x)))
  else: 
    return sigmoid(x) * (1 - sigmoid(x))
grad_ad = np.vectorize(ad.grad(sigmoid))
grad_fd = nd.Derivative(sigmoid, n=1)

fig_kw = dict(width=200, height=150)
dom = np.linspace(-5, 5, 1000)
p = figure(**fig_kw, title="Original function"); p.line(dom, sigmoid(dom))
q = figure(**fig_kw, title="Closed-form"); q.line(dom, sigmoid(dom, derivative=True), color='red')
r = figure(**fig_kw, title="Automatic"); r.line(dom, grad_ad(dom), color='green')
s = figure(**fig_kw, title="Finite-difference"); s.line(dom, grad_fd(dom), color="purple")
show(row(p, q, r, s))


# Decomposing the computation

For a fixed simplicial complex $K$ of size $\lvert K \rvert = N$, each term of our function to maximize $\mu_R(\tau)$ is a composite of several functions:

$$ \mathcal{m}(\tau) = \tau \overset{f}{\mapsto} f_\tau(K) \overset{g}{\mapsto} \mathcal{L}_{p}^{\text{up}} \overset{h}{\mapsto} \Phi_\epsilon(\mathcal{L}_{p}^{\text{up}}) \overset{\eta}{\mapsto}\lVert \Phi_\epsilon(\mathcal{L}_{p}^{\text{up}}) \rVert_{\ast} $$

where the domain/codomain of each function in the composition is given by: 
$$
\begin{align*}
  f& :& \mathbb{R} & \to \mathbb{R}^{N} \\
  g& :& \mathbb{R}^{N} & \to \mathbb{R}^{N \times N} \\
  h& :& \mathbb{R}^{N \times N} & \to \mathbb{R}^{N \times N} \\
  \eta& :& \mathbb{R}^{N \times N} & \to \mathbb{R}_+  
\end{align*}
$$

Just as the scalar-product $f_\tau(K)$ equipped to $K$ can be thought of as relation $(\sigma, f_\tau(\sigma))$ defined over every $\sigma \in K$, the map from the scalar-product to the combinatorial (up) Laplacian $g$ can also be thought of as a relation: 

$$ \big ( (i,j), ... \big )$$

$$ 
\mathcal{L}_{p}^{\text{up}} = \mathcal{L}_{p}^{\text{up}}(f_\tau(K)) = D_{p}^{+/2} \partial_{p+1} W_{p+1} \partial_{p+1}^T D_{p}^{+/2}
$$


The chain rule says that evaluating the gradient for this composition requires: 


For this we can just use the proximal gradient algorithm. 

The linear operator $F: S^m \to S^m$ is called a Löwner operator if there exists a function $f : \mathbb{R} \to \mathbb{R}$ such that: 
$$ F(A) = U \mathrm{Diag}(f(\lambda_1(A)), f(\lambda_2(A)), \dots, f(\lambda_m(A))) U^T$$
where $U \in O^m$ is an orthonormal matrix diagonalizing $A = U \Lambda U^T$. 

Theorem V.3.3 of [Matrix Analyis](https://d1wqtxts1xzle7.cloudfront.net/34000331/_Rajendra_Bhatia__Matrix_Analysis_%28Graduate_Texts_%28bookos-z1.org%29.pdf?1403311534=&response-content-disposition=inline%3B+filename%3DGeneral_Theory.pdf&Expires=1677700382&Signature=HW~EqW60~Q6ukWMF9zbsoqMVYDOhDeQBVf5KkMUkmYZC37D9HFydsOi3XjOJ~K6~2q~bDGB81mw0vyluMDPJr6abtTDyi4wB2wmut-3t6P-nyO9Me~v6QOPYi-orINMJAS3nVr965z3JZYJSyTisHCDGb4k566nfWGlrYrukQp-fEf4kucpQXg46EqAWlDL4otWGQlz5~nbv61C6MK7vquozSvQtgePWDNMd9M4wSZsNV1KUwrnITBsV9qpXg45iy6HzA9f3cYiiaqPZ3Q4xX2LxtKLLafzRt2Q2r9oGuur1JXscvOKvQdKHLVodHG-khkd4swnVGg5dgDBhiaueIw__&Key-Pair-Id=APKAJLOHF5GGSLRBV4ZA) characterizes the differential of the map $h$. Namely, for the pair $(F, f)$ where $F$ is a Löwner operator and $f \in C^{1}(I)$ a function defined over the interval $f: I \to \mathbb{R}$, the differential of $F$ is completely characterized by in terms of the divided differences of $f$. In particular, if $A$ a Hermitian matrix with all eigenvalues in $I$, the differential $\nabla F(A)[H]$ of $f$ at $A$ is given by: 

$$ \nabla F(A)[H] = U(f^{[1]}(\Lambda(A)) \circ U^T H U ) U^T  $$
<!-- $$ Df(A)(H) = f^{[1]}(A) \circ H $$ -->

where $H \in S_m$ denotes any Hermitian matrix, $\circ$ denotes the Hadamard product, and $f^{[1]}(x)$ denotes the _divided difference_ matrix of a given vector $x \in \mathbb{R}^m$: 

$$ 
\big(f^{[1]}(x) \big)_{ij} = \begin{cases}
f'(x_i) & x_i = x_j \\
\frac{f(x_i) - f(x_j)}{x_i - x_j} & x_i \neq x_j
\end{cases}
$$

# Defining the functions

In [4]:
from scipy.sparse import spmatrix, diags
from splex import * 
import hirola
from pbsig.linalg import up_laplacian
X = np.random.uniform(size=(5,2)) # point cloud
fw = flag_weight(X)
S = simplicial_complex([[0,1,2,3,4]])

## Use linearity to simplify f! 
f = lambda t: np.array([t*fw(s) for s in S])

## Save base-laplacian complex non-zoer indices for fixed 'S' to always yield 
## fixed vector result. Could optimize in theory to be matrix-free (but will
## always return result proportional to number of non-zeros in up-Laplacian)
def make_g(S: ComplexLike):
  L = up_laplacian(S, p=1, normed=True)
  I,J = L.nonzero()
  ht = hirola.HashTable(int(len(I)*1.25), dtype=(np.uint16, 2))
  ht.add(np.c_[I,J].astype(np.uint16))
  data_vec = L.data.copy()
  def _g(f: Union[Callable, ArrayLike], as_matrix: bool = False):
    data_vec.fill(0)
    L_tmp = up_laplacian(S, p=1, weight=f)
    data_vec[ht[np.c_[L_tmp.nonzero()].astype(np.uint16)]] = L_tmp.data
    if as_matrix: 
      L.data = data_vec
      return L
    else:
      return data_vec
  return _g
g = make_g(S)

## Tackle h! 
## Simple way first: just finite-difference it up! 
from pbsig.linalg import sgn_approx, eigh_solver
sa = sgn_approx(eps=1e-1, p=1.5)
def h(X: spmatrix) -> ArrayLike:
  from scipy.sparse import diags
  ew, ev = eigh_solver(X)(X)
  return ev @ diags(sa(ew)) @ ev.T # note this is dense ndarray 

## normalized laplacian is scale invariant so this wont work directly 
# h(g(f(0.1), as_matrix=True)) - h(g(f(0.11), as_matrix=True))

# since eigenvalues of psd are singular values 
eta = lambda X: sum(abs(np.linalg.eigvalsh(X)))

## The objective function 
print(eta(h(g(f(0.1), as_matrix=True))))

5.988949226564851


## Defining their derivatives via finite-differences

We have options for getting at the derivatives. 

Chain rule says if the gradient of our composite objective $(\eta \circ h \circ g \circ f)(\tau)$ is given by:

$$ m'(\tau) = (\eta' \circ h \circ g \circ f)(\tau) \cdot (h' \circ g \circ f)(\tau) \cdot (g' \circ f)(\tau) \cdot f'(\tau) $$

In [ ]:

## Step one: parameterize tau -> Laplacian 
# Start by form the fixed (constant) boundary matrix 
S = simplicial_complex([[0,1,2]])
D = boundary_matrix(S, p=1)

# Then parameterize the scaling 
X = np.random.uniform(size=(3,2))
np.diag() @ D @ np.diag() @ D.T @ np.diag()
E = [[0,1], [0,2], [1,2]]
f = lambda s: max(s)*5.0
h = lambda t: np.repeat()


X = np.random.uniform(size=(10,2))
X = X @ X.T
sum(np.linalg.eigvalsh(X))
sum(np.linalg.eigvalsh(X)/(np.linalg.eigvalsh(X)+0.01))

sum(np.linalg.eigvalsh(X / (X + 0.01)))


B = boundary_matrix(S, p = p+1)
if normed: 
  L = B @ diags(wq) @ B.T
  deg = L.diagonal()
  L = (diags(pseudo(np.sqrt(deg))) @ L @ diags(pseudo(np.sqrt(deg)))).tocoo()

# eps, p = 1e-1, 1.5
# phi = lambda x: ad.sign(x) * (ad.abs(x)**p / (ad.abs(x)**p + eps**p))
# grad_phi = ad.grad(phi)

## need gradient of proximal 
## and gradient via finite diffference of laplacians 

# Gradient ascent 

Our function to optimize $\mu_R(\tau)$ is a composite of two functions $\mu_R(\tau) = (f \circ g \circ h)(\tau) = f(g(h(\tau)))$, where $f$, $g$, and $h$ are given by:

$$ \tau \overset{h}{\mapsto} (\sigma, f_\tau(\sigma)) \mapsto \mathcal{L}_{p+1}^{\text{up}} \overset{g}{\mapsto} \Phi_\epsilon(\mathcal{L}_{p+1}^{\text{up}}) \overset{f}{\mapsto}\lVert \Phi_\epsilon(\mathcal{L}_{p+1}^{\text{up}})  \rVert_{\ast} $$

where:

$$
\begin{align}
  h& :& \mathbb{R} & \to \mathbb{R}^{m \times m} \\
  g& :& \mathbb{R}^{m \times m} & \to \mathbb{R}^{m \times m} \\
  f& :& \mathbb{R}^{m \times m} & \to \mathbb{R}_+  
\end{align}
$$
The chain rule says that evaluating the gradient for this composition requires: 

$$ f'(g(h(\tau))) \cdot g'(h(\tau)) \cdot h'(\tau)$$


For this we can just use the proximal gradient algorithm. 

In [ ]:
# simple_f = lambda x: x**2
# simple_g = lambda x, t: t*simple_f(x)

# dom = np.linspace(-2, 2, 100)
# p = figure()
# p.line(dom, simple_f(dom), color='black')
# p.line(dom, simple_g(dom, 2), color='red')
# show(p)
